In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

In [18]:
df=pd.read_csv('feature_time_48k_2048_load_1.csv')
df.columns

feature_df=df[['max', 'min', 'mean', 'sd', 'rms', 'skewness', 'kurtosis', 'crest',
       'form',]]
X=np.asarray(feature_df)
y=np.asarray(df['fault'])


In [30]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=42)
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [ ]:
%pip install optuna

In [31]:
import lightgbm as lgb
from lightgbm import early_stopping


In [55]:
import optuna
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
from sklearn.metrics import accuracy_score
import numpy as np

def objective(trial):
    params = {
        'objective': 'multiclass',
        'num_class': len(np.unique(y_train)),
        'metric': 'multi_logloss',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
        'num_leaves': trial.suggest_int('num_leaves', 31, 256),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 20, 100),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
    }

    # Ensure that X_train, y_train, X_test, y_test are accessible here
    gbm = lgb.train(
        params,
        lgb.Dataset(X_train, label=y_train),
        num_boost_round=200,
        valid_sets=[lgb.Dataset(X_test, label=y_test)],
        callbacks=[early_stopping(stopping_rounds=20), log_evaluation(10)]  # Verbose every 10 iterations
    )

    preds = gbm.predict(X_test, num_iteration=gbm.best_iteration)
    pred_labels = np.argmax(preds, axis=1)
    accuracy = accuracy_score(y_test, pred_labels)
    return 1 - accuracy  # Minimize the error

# Now run the optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print('Best trial:')
trial = study.best_trial
print(f'  Accuracy: {1 - trial.value}')
print('  Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')


[I 2024-10-05 14:58:04,138] A new study created in memory with name: no-name-a09b5575-32f0-4bd8-a410-ed1098307456
C:\Users\nilarghya\AppData\Local\Temp\ipykernel_19804\1114577306.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
C:\Users\nilarghya\AppData\Local\Temp\ipykernel_19804\1114577306.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
C:\Users\nilarghya\AppData\Local\Temp\ipykernel_19804\1114577306.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://git

Training until validation scores don't improve for 20 rounds
[10]	valid_0's multi_logloss: 1.71084
[20]	valid_0's multi_logloss: 1.34926
[30]	valid_0's multi_logloss: 1.10578
[40]	valid_0's multi_logloss: 0.934154
[50]	valid_0's multi_logloss: 0.799593
[60]	valid_0's multi_logloss: 0.697662
[70]	valid_0's multi_logloss: 0.61514
[80]	valid_0's multi_logloss: 0.552223
[90]	valid_0's multi_logloss: 0.502547
[100]	valid_0's multi_logloss: 0.459984
[110]	valid_0's multi_logloss: 0.426221
[120]	valid_0's multi_logloss: 0.397132
[130]	valid_0's multi_logloss: 0.374055
[140]	valid_0's multi_logloss: 0.354964
[150]	valid_0's multi_logloss: 0.338816
[160]	valid_0's multi_logloss: 0.325488
[170]	valid_0's multi_logloss: 0.314208
[180]	valid_0's multi_logloss: 0.304693
[190]	valid_0's multi_logloss: 0.29758
[200]	valid_0's multi_logloss: 0.291518
Did not meet early stopping. Best iteration is:
[200]	valid_0's multi_logloss: 0.291518


[I 2024-10-05 14:58:06,399] Trial 0 finished with value: 0.07826086956521738 and parameters: {'learning_rate': 0.014951631690659953, 'num_leaves': 48, 'max_depth': 10, 'min_data_in_leaf': 66, 'lambda_l1': 6.098940547543696e-07, 'lambda_l2': 2.41851459842098, 'feature_fraction': 0.8410784813228489, 'bagging_fraction': 0.6298337336419392, 'bagging_freq': 7}. Best is trial 0 with value: 0.07826086956521738.
C:\Users\nilarghya\AppData\Local\Temp\ipykernel_19804\1114577306.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
C:\Users\nilarghya\AppData\Local\Temp\ipykernel_19804\1114577306.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use su

Training until validation scores don't improve for 20 rounds
[10]	valid_0's multi_logloss: 1.89642
[20]	valid_0's multi_logloss: 1.61833
[30]	valid_0's multi_logloss: 1.41104
[40]	valid_0's multi_logloss: 1.24809
[50]	valid_0's multi_logloss: 1.11639
[60]	valid_0's multi_logloss: 1.00731
[70]	valid_0's multi_logloss: 0.915019
[80]	valid_0's multi_logloss: 0.835275
[90]	valid_0's multi_logloss: 0.768352
[100]	valid_0's multi_logloss: 0.709529
[110]	valid_0's multi_logloss: 0.657332
[120]	valid_0's multi_logloss: 0.611236
[130]	valid_0's multi_logloss: 0.570389
[140]	valid_0's multi_logloss: 0.534317
[150]	valid_0's multi_logloss: 0.503638
[160]	valid_0's multi_logloss: 0.475605
[170]	valid_0's multi_logloss: 0.449523
[180]	valid_0's multi_logloss: 0.427207
[190]	valid_0's multi_logloss: 0.40756
[200]	valid_0's multi_logloss: 0.389663
Did not meet early stopping. Best iteration is:
[200]	valid_0's multi_logloss: 0.389663


[I 2024-10-05 14:58:08,575] Trial 1 finished with value: 0.08478260869565213 and parameters: {'learning_rate': 0.00814919856503571, 'num_leaves': 207, 'max_depth': 11, 'min_data_in_leaf': 67, 'lambda_l1': 0.04384503066520351, 'lambda_l2': 7.042299900590663e-06, 'feature_fraction': 0.7836445727924901, 'bagging_fraction': 0.6040069991114005, 'bagging_freq': 1}. Best is trial 0 with value: 0.07826086956521738.
C:\Users\nilarghya\AppData\Local\Temp\ipykernel_19804\1114577306.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
C:\Users\nilarghya\AppData\Local\Temp\ipykernel_19804\1114577306.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use

...

[80]	valid_0's multi_logloss: 0.260695
Early stopping, best iteration is:
[68]	valid_0's multi_logloss: 0.258522


C:\Users\nilarghya\AppData\Local\Temp\ipykernel_19804\1114577306.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
C:\Users\nilarghya\AppData\Local\Temp\ipykernel_19804\1114577306.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
C:\Users\nilarghya\AppData\Local\Temp\ipykernel_19804\1114577306.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda_l2': trial.suggest_

Training until validation scores don't improve for 20 rounds
[10]	valid_0's multi_logloss: 1.03866
[20]	valid_0's multi_logloss: 0.643976
[30]	valid_0's multi_logloss: 0.457585
[40]	valid_0's multi_logloss: 0.35803
[50]	valid_0's multi_logloss: 0.305111
[60]	valid_0's multi_logloss: 0.276942
[70]	valid_0's multi_logloss: 0.259842
[80]	valid_0's multi_logloss: 0.251688
[90]	valid_0's multi_logloss: 0.252367


[I 2024-10-05 15:00:38,359] Trial 99 finished with value: 0.07173913043478264 and parameters: {'learning_rate': 0.04102463252362221, 'num_leaves': 233, 'max_depth': 10, 'min_data_in_leaf': 42, 'lambda_l1': 0.00011082714498874595, 'lambda_l2': 1.0573179587530582e-08, 'feature_fraction': 0.7992057670795863, 'bagging_fraction': 0.6453419177703942, 'bagging_freq': 3}. Best is trial 5 with value: 0.06521739130434778.


[100]	valid_0's multi_logloss: 0.252061
Early stopping, best iteration is:
[85]	valid_0's multi_logloss: 0.249945
Best trial:
  Accuracy: 0.9347826086956522
  Params: 
    learning_rate: 0.053579200419262236
    num_leaves: 164
    max_depth: 3
    min_data_in_leaf: 60
    lambda_l1: 0.0004248085758851102
    lambda_l2: 2.365336284824762e-08
    feature_fraction: 0.9536606109774236
    bagging_fraction: 0.6708056640477734
    bagging_freq: 1


import pywt

def extract_features_wpd(signal):
    wp = pywt.WaveletPacket(data=signal, wavelet='db4', mode='symmetric', maxlevel=5)
    nodes = wp.get_level(5)
    features = []
    for node in nodes:
        coeff = node.data
        # Extract statistical features
        features.extend([
            np.mean(coeff),
            np.std(coeff),
            np.max(coeff),
            np.min(coeff),
            np.sum(np.abs(coeff)),
            np.sqrt(np.mean(coeff ** 2))  # Root mean square
        ])
    return features

# Apply to dataset
X_train = np.array([extract_features_wpd(sig) for sig in X_train])
X_test = np.array([extract_features_wpd(sig) for sig in X_test])










































































































































































































































m,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,